## Module 6 Asignment (using Sframes)
# Decision Trees in Practice

In this assignment we will explore various techniques for preventing overfitting in decision trees. We will extend the implementation of the binary decision trees that we implemented in the previous assignment. You will have to use your solutions from this previous assignment and extend them.

In this assignment you will:

* Implement binary decision trees with different early stopping methods.
* Compare models with different stopping parameters.
* Visualize the concept of overfitting in decision trees.

Import SFrames.

In [46]:
import sframe as sf

# Load LendingClub Dataset

**1.** This assignment will use the [LendingClub](https://www.lendingclub.com/) dataset used in the previous two assignments.

In [47]:
loans = sf.SFrame('../Week3/lending-club-data.gl/')

**2.** As before, we reassign the labels to have +1 for a safe loan, and -1 for a risky (bad) loan.

In [48]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

**3.** We will be using the same 4 categorical features as in the previous assignment: 
1. grade of the loan 
2. the length of the loan term
3. the home ownership status: own, mortgage, rent
4. number of years of employment.

In the dataset, each of these features is a categorical feature. Since we are building a binary decision tree, we will have to convert this to binary data in a subsequent section using 1-hot encoding.

In [49]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

## Subsample dataset to make sure classes are balanced

**4.** Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We used `seed = 1` so everyone gets the same results.

In [50]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Transform categorical data into binary features

Since we are implementing **binary decision trees**, we transform our categorical data into binary data using 1-hot encoding, just as in the previous assignment. Here is the summary of that discussion:

For instance, the **home_ownership** feature represents the home ownership status of the loanee, which is either `own`, `mortgage` or `rent`. For example, if a data point has the feature 
```
   {'home_ownership': 'RENT'}
```
we want to turn this into three features: 
```
 { 
   'home_ownership = OWN'      : 0, 
   'home_ownership = MORTGAGE' : 0, 
   'home_ownership = RENT'     : 1
 }
```

Since this code requires a few Python and GraphLab tricks, feel free to use this block of code as is. Refer to the API documentation for a deeper understanding.

**5.** This technique of turning categorical variables into binary variables is called one-hot encoding. Using the software of your choice, perform one-hot encoding on the four features described above. You should now have 25 binary features.


In [51]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

The feature columns now look like this:

In [52]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

## Train-Validation split

**6.** We split the data into a train-validation split with 80% of the data in the training set and 20% of the data in the validation set. We use `seed=1` so that everyone gets the same result.

In [53]:
train_data, validation_set = loans_data.random_split(.8, seed=1)

print 'Training set   : %d data points' % len(train_data)
print 'Test set       : %d data points' % len(validation_set)

Training set   : 37224 data points
Test set       : 9284 data points


# Early stopping methods for decision trees

In this section, we will extend the **binary tree implementation** from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

1. Reached a **maximum depth**. (set by parameter `max_depth`).
2. Reached a **minimum node size**. (set by parameter `min_node_size`).
3. Don't split if the **gain in error reduction** is too small. (set by parameter `min_error_reduction`).

For the rest of this assignment, we will refer to these three as **early stopping conditions 1, 2, and 3**.

## Early stopping condition 1: Maximum depth

Recall that we already implemented the maximum depth stopping condition in the previous assignment. In this assignment, we will experiment with this condition a bit more and also write code to implement the 2nd and 3rd early stopping conditions.

We will be reusing code from the previous assignment and then building upon this.  We will **alert you** when you reach a function that was part of the previous assignment so that you can simply copy and past your previous code.

## Early stopping condition 2: Minimum node size

The function **reached_minimum_node_size** takes 2 arguments:

1. The `data` (from a node)
2. The minimum number of data points that a node is allowed to split on, `min_node_size`.

**7.** This function simply calculates whether the number of data points at a given node is less than or equal to the specified minimum node size. This function will be used to detect this early stopping condition in the **decision_tree_create** function. Thus if a node being considered for further split has fewer than `min_node_size`, then it won't be split further, instead it wil be converted to a leaf.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below.

In [83]:
def reached_minimum_node_size(data, min_node_size):
    # Return True if the number of data points is less than or equal to the minimum node size.
    ## YOUR CODE HERE
     return  ( len(data) <= min_node_size)

** Quiz question:** Given an intermediate node with 6 safe loans and 3 risky loans, if the `min_node_size` parameter is 10, what should the tree learning algorithm do next? 6+3=9. Given number of points in the node `len(data)=9` is less than the `min_node_size=10` , we wont consider this node for further split, instead convert it to a leaf.

In [87]:
#if  reached_minimum_node_size(data, min_node_size):         ## YOUR CODE HERE 
#        print "Early stopping condition 2 reached. Reached minimum node size."
#        return create_leaf(target_values)  ## YOUR CODE HERE

## Early stopping condition 3: Minimum gain in error reduction

The function **error_reduction** takes 2 arguments:

1. The error **before** a split, `error_before_split`.
2. The error **after** a split, `error_after_split`.

**8.** This function computes the gain in error reduction, i.e., the difference between the error before the split and that after the split. This function will be used to detect this early stopping condition in the **decision_tree_create** function.

Fill in the parts of the function below where you find `## YOUR CODE HERE`.  There is **one** instance in the function below. 

In [55]:
def error_reduction(error_before_split, error_after_split):
    # Return the error before the split minus the error after the split.
    return (error_before_split-error_after_split)

** Quiz question:** Assume an intermediate node has 6 safe loans and 3 risky loans.  For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the **minimum gain in error reduction** parameter is set to 0.2, what should the tree learning algorithm do next? 

We can see that, even with the best splitting feature, the error reduction is at most only 0.14; this is less than the target `min_error_reduction=0.2`. So there is no point splitting the intermediate node. Set it as leaf and return.        

In [88]:
#error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))

# error_b4_split = mistakes=3/len(3+6) = 3/9 = 1/3 = 0.33

#if  error_reduction(error_before_split, error_after_split) <= min_error_reduction: 
#        print "Early stopping condition 3 reached. Minimum error reduction."      
#        return create_leaf(target_values)
#max error reduction = 0.14 = 0.33 - 0.9, 0.14 <= 0.2, 
#so not quite achieving as much error reduction as we want, so no point splitting further! 

## Grabbing binary decision tree helper functions from past assignment

**9.** Recall from the previous assignment that we wrote a function `intermediate_node_num_mistakes` that calculates the number of **misclassified examples** when predicting the **majority class**. This is used to help determine which feature is best to split on at a given node of the tree.

**Paste your code for `intermediate_node_num_mistakes` here**.

In [56]:
def intermediate_node_num_mistakes(labels_in_node):
    
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # print "Number of samples %d " %len(labels_in_node)
    # print "Samples class distribution %s " %(labels_in_node)
    # mistakes = None
    
    # Count the number of 1's (safe loans)
    # num_of_1s =      (loans['safe_loan'] == +1).sum()
    safe_loans_count = sum(labels_in_node == +1)
    # safe_loans_count = sum([1 if x == 1 else 0 for x in labels_in_node])
    
    # Count the number of -1's (risky loans)
    risky_loans_count = sum(labels_in_node == -1)
    # risky_loans_count = sum([1 if x == -1 else 0 for x in labels_in_node])
       
    
    ##uncomment if want to print some diagnostics...
    
    # print "(num risky: %d , num safe: %d) in the split." %(risky_loans_count,safe_loans_count)
    
    # Return the number of mistakes that the majority classifier makes.
    #maj_class = '+1' if safe_loans_count >= risky_loans_count else '-1'
    
    
    #mistakes = risky_loans_count if (maj_class == '+1') else safe_loans_count
    #mistakes = risky_loans_count if (safe_loans_count >= risky_loans_count) else safe_loans_count
    # what above line is doing is just trying to find minimum of the two count values!!!
    # hint
    # Since we are assuming majority class prediction, 
    # all the data points that are not in the majority class are considered mistakes
    # we don't even have to determine majority class
    # the bigger count will always be majority, lower count wud always be mistakes
    # mistakes = min(safe_loans_count, risky_loans_count)   #as easy as that!!
    # print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    
    # So for e.g if we have more safe loans than risky, majority vote wud be to classify the
    # whole lot as of safe class, and mistakes would be just count of the other (risky) class
    # so just return count of the smaller class sample, cause that wud always be minority class
    # and minority class always be misclassfied !!    
    
    '''
    # uncomment if want to print extra some diagnostics... 
    if (maj_class == '+1'):
        #mistakes = len(labels_in_node) - safe_loans_count
        mistakes = risky_loans_count
        print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    else:
        #mistakes = len(labels_in_node) - risky_loans_count
        mistakes = safe_loans_count
        print "Majority class is %s so num mistakes will be %d " %(maj_class, mistakes)
    '''  
    return min(safe_loans_count, risky_loans_count)

**10.** We also wrote a function `best_splitting_feature` that finds the best feature to split on given the data and a list of features to consider.

**Paste your `best_splitting_feature` code here**.

In [57]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    # print "num_data_points = %d" %(num_data_points)
    # print data[features].head(1)
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        #print "\n************Trying out feature:", feature
        
        # The left split will have all data points where the feature value is 0
        left_split = data[ data[feature] == 0 ]
        #print "left split",left_split[feature] #this will just split out a list of 0's
        #print "num_data_points in (data[%s] == 0) i.e. left_split = %d " %(feature,len(left_split))
        
        # The right split will have all data points where the feature value is 1
        right_split =  data[ data[feature] == 1 ]
        #print "right split", right_split[feature] #this will just split out a list of 1's
        #print "num_data_points (data[%s] == 1) i.e. right_split = %d " %(feature,len(right_split))
        
        
        # Calculate the number of misclassified examples in the left split.
        left_mistakes = intermediate_node_num_mistakes((left_split[target]))
        
        # What we are doing above is is trying to find out how many target class is +ve 
        # and how many target class is -ve for cases where the feature values are 0
        # so we try find majority class label where the feature values are all 0
        # Reuse function intermediate_node_num_mistakes()
        # this function determines how many samples data[ data[feature] == 0 ]
        # have +ve labels and how many -ve lables, then determines majority lable
        # say if majority were +ves, then mistakes is simply count of -ves in sample
        
             
        # Calculate the number of misclassified examples in the right split.
        right_mistakes = intermediate_node_num_mistakes((right_split[target]))
        
        # Now repeat above step for all data points where data[ data[feature] == 1 ] 
        # i.e where the feature variable in question has all values 1
        # and find number of mistakes here same way as above (find majority class
        # and mistakes is just count of data points of the other minority class)
        
        #print num_data_points == (len(left_split) + len(right_split))                  
        
            
        # -------------------------------------------------------------------------    
        # Compute the classification error of this split.
        # i.e if we chose this feature to make a decision/split in tree 
        # then how do we measure how pure leaf nodes we will get
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        error = (left_mistakes + right_mistakes)/num_data_points
        #print "Error using %s for split is %0.4f " %(feature, error)
        
        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        if error < best_error:
            print "\nFOUND LOWER ERROR using %s, error is %0.4f, previous best error %0.4f" \
                                             %(feature, error, best_error)
            best_feature = feature
            best_error = error
            
    
    return best_feature # Return the best feature we found

*11.** Finally, recall the function `create_leaf` from the previous assignment, which creates a leaf node given a set of target values.  

**Paste your `create_leaf` code here**.

In [58]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None, # unknown at this stage
            'left' : None,              # unknown at this stage
            'right' : None,             # unknown at this stage
            'is_leaf': True     #set to True cause we are creating a leaf
           }  
    
    # Count the number of data points that are +1 and -1 in this node.
    num_plus_ones =      len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_plus_ones > num_minus_ones:
        leaf['prediction'] =  +1        ## YOUR CODE HERE
    else:
        leaf['prediction'] =  -1        ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

## Incorporating new early stopping conditions in binary decision tree implementation

**12.** Now, you will implement a function that builds a decision tree handling the three early stopping conditions described in this assignment.  In particular, you will write code to detect early stopping conditions 2 and 3.  You implemented above the functions needed to detect these conditions.  The 1st early stopping condition, **max_depth**, was implemented in the previous assigment and you will not need to reimplement this.  In addition to these early stopping conditions, the typical stopping conditions of having no mistakes or no more features to split on (which we denote by "stopping conditions" 1 and 2) are also included as in the previous assignment.

**Implementing early stopping condition 2: minimum node size:**

* **Step 1:** Use the function **reached_minimum_node_size** that you implemented earlier to write an if condition to detect whether we have hit the base case, i.e., the node does not have enough data points and should be turned into a leaf. Don't forget to use the `min_node_size` argument.
* **Step 2:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.


**Implementing early stopping condition 3: minimum error reduction:**

**Note:** This has to come after finding the best splitting feature so we can calculate the error after splitting in order to calculate the error reduction.

* **Step 1:** Calculate the **classification error before splitting**.  Recall that classification error is defined as:

$$
\text{classification error} = \frac{\text{# mistakes}}{\text{# total examples}}
$$
* **Step 2:** Calculate the **classification error after splitting**. This requires calculating the number of mistakes in the left and right splits, and then dividing by the total number of examples.
* **Step 3:** Use the function **error_reduction** to that you implemented earlier to write an if condition to detect whether  the reduction in error is less than the constant provided (`min_error_reduction`). Don't forget to use that argument.
* **Step 4:** Return a leaf. This line of code should be the same as the other (pre-implemented) stopping conditions.

Fill in the places where you find `## YOUR CODE HERE`. There are **seven** places in this function for you to fill in.

In [59]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10, min_node_size=1, 
                         min_error_reduction=0.0):
    
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    
    
    # STOPPING CONDITION 1: All nodes are of the same type.
    # (Check if there are NO or ZERO mistakes at current node.
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached. No mistakes - PURE node!"                
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)

    # STOPPING CONDITION 2: No more features to split on.
    # (check if there are remaining features to consider splitting on)
    # if len(remaining_features) is  0 then no features left in list    
    if remaining_features == []: #if remaining_features == 0:
        print "Stopping condition 2 reached. No remaining splitting features."   
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # EARLY STOPPING CONDITION 1: Reached max depth limit.
    if current_depth >= max_depth:
        print "Early stopping condition 1 reached. Reached maximum depth."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    # EARLY STOPPING CONDITION 2: Reached the minimum node size.
    # If the number of data points is less than or equal to the minimum size, return a leaf.
    if  reached_minimum_node_size(data, min_node_size):         ## YOUR CODE HERE 
        print "Early stopping condition 2 reached. Reached minimum node size."
        return create_leaf(target_values)  ## YOUR CODE HERE
    
    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    # we assume that this fn will always find n return a feature - so no error checking
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    # EARLY STOPPING CONDITION 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples 
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting (number of misclassified examples 
    # in both groups divided by the total number of examples)
    left_mistakes = intermediate_node_num_mistakes(left_split[target])   ## YOUR CODE HERE
    right_mistakes =intermediate_node_num_mistakes(right_split[target])    ## YOUR CODE HERE
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if  error_reduction(error_before_split, error_after_split) <= min_error_reduction:   ## YOUR CODE HERE
        print "Early stopping condition 3 reached. Minimum error reduction."
        return create_leaf(target_values)   ## YOUR CODE HERE 
    
    
    remaining_features.remove(splitting_feature)  #remove feature we split on
    #print "\n\nSPLIT ON FEATURE that gives lowest error -> %s. (Left = %s, Right = %s)" % (\
    #                  splitting_feature, len(left_split), len(right_split))
    
    
    # Create a leaf node if the split is "perfect" - not happen often!
    # Not checking for these - good - so we can create more pure nodes!!!
    '''
    if len(left_split) == len(data):
        print "CREATING A PURE LEFT LEAF NODE."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "CREATING A PURE RIGHT LEAF NODE"
        return create_leaf(right_split[target])    
    '''
    
    # Now enter recursive binary splitting -  a top-down, greedy approach
    # Start at top of the tree (point at which all observations belong to a single region) 
    # and then successively splits the predictor space; 
    # each split is indicated via two new branches further down on the tree    
    # remaining_features won't consider feature we split on above,
    # inc depth as we going one level deeper, target is just 'soft_loans'
    print "########### Growing left branch of tree further ##############"
    # Takes dataframe where value for predictor_feature (splitting_feature) is 0 and
    # subset of features and grows decision tree on that branch    
    
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        
    
    
    print "########### Growing right branch of tree further ##############"
    # Takes dataframe where value for predictor_feature (splitting_feature) is 1 and
    # subset of features and grows decision tree on that branch        
    
    ## YOUR CODE HERE
    right_tree = decision_tree_create(right_split, remaining_features, target, 
                                     current_depth + 1, max_depth, min_node_size, min_error_reduction)        

    # now we have two dictionary objects left_tree and right_tree
    # so node we are at can't be a leaf, cause we have left and right branches - is_leaf FALSE
    # also store the left_tree and right_tree dictionaries at this node
    # all done in the return stmnt BLW
    # da magic of recursion happens in the return statement !!  
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

Here is a function to count the nodes in your tree:

In [60]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

Run the following test code to check your implementation. Make sure you get **'Test passed'** before proceeding.

In [89]:
small_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 2, 
                                        min_node_size = 10, min_error_reduction=0.0)
if count_nodes(small_decision_tree) == 7:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_decision_tree)
    print 'Number of nodes that should be there : 7' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Growing right branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Grow

## Build a tree!

**13.** Now that your code is working, we will train a tree model on the **train_data** with
* `max_depth = 6`
* `min_node_size = 100`, 
* `min_error_reduction = 0.0`

**Warning**: This code block may take a minute to learn. Call this model my_decision_tree_new.

In [94]:
my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 100, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
Early stopping condition 3 reached. Minimum error reduction.
########### Growing right branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (101 data poi

**14.** Let's now train a tree model **ignoring early stopping conditions 2 and 3** so that we get the same tree as in the previous assignment.  To ignore these conditions, we set `min_node_size=0` and `min_error_reduction=-1` (a negative value).Call this model my_decision_tree_old. 

In [95]:
my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                                min_node_size = 0, min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

## Making predictions

**15.** Recall that in the previous assignment we implemented a function `classify` to classify a new point `x` using a given `tree`.

**Paste your `classify` code here**.

In [96]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

**16.** Now, let's consider the first example of the validation set and see what the `my_decision_tree_new` model predicts for this data point.

In [97]:
print validation_set[0]
print '\nPredicted class: %s ' % classify(my_decision_tree_new, validation_set[0])

{'emp_length.3 years': 0, 'home_ownership.RENT': 1, 'home_ownership.OWN': 0, 'emp_length.6 years': 0, 'emp_length.9 years': 0, 'emp_length.1 year': 0, 'home_ownership.OTHER': 0, 'safe_loans': -1, 'emp_length.< 1 year': 0, 'emp_length.10+ years': 0, 'emp_length.5 years': 0, 'term. 60 months': 1, 'home_ownership.MORTGAGE': 0, 'emp_length.2 years': 1, 'emp_length.7 years': 0, 'emp_length.n/a': 0, 'grade.D': 1, 'grade.E': 0, 'grade.F': 0, 'grade.G': 0, 'grade.A': 0, 'grade.B': 0, 'grade.C': 0, 'emp_length.4 years': 0, 'term. 36 months': 0, 'emp_length.8 years': 0}

Predicted class: -1 


**17.** Let's add some annotations to our prediction to see what the prediction path was that lead to this predicted class:

In [98]:
classify(my_decision_tree_new, validation_set[0], annotate = True)

Split on term. 36 months = 0
Split on grade.A = 0
At leaf, predicting -1


-1

**18.** Let's now recall the prediction path for the decision tree learned in the previous assignment, which we recreated here as `my_decision_tree_old`.

In [99]:
classify(my_decision_tree_old, validation_set[0], annotate = True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
Split on grade.E = 0
At leaf, predicting -1


-1

* my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6, 
                               min_node_size = 100, min_error_reduction=0.0)
* my_decision_tree_old = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6,
                                min_node_size = 0, min_error_reduction=-1)

** Quiz question:** For `my_decision_tree_new` trained with `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, is the prediction path for `validation_set[0]` shorter, longer, or the same as for `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?  

Much Shorter - only 2 splits.

**Quiz question:** For `my_decision_tree_new` trained with `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, is the prediction path for **any point** always shorter, always longer, always the same, shorter or the same, or longer or the same as for `my_decision_tree_old` that ignored the early stopping conditions 2 and 3?

Should always be shorter. **any point** bit tricky, maybe `shorter or the same` is better!!

** Quiz question:** For a tree trained on **any** dataset using `max_depth = 6`, `min_node_size = 100`, `min_error_reduction=0.0`, what is the maximum number of splits encountered while making a single prediction?

assuming dataset large enough so min_node_size=100 not a huge constraint, and suppose  `min_error_reduction=0.0` is satisfied as well. we should be able to grow a tree upto max-depth 6, so at most max 6 splits!!!

## Evaluating the model

**19.** Now let us evaluate the model that we have trained. You implemented this evautation in the function `evaluate_classification_error` from the previous assignment.

**Please copy and paste your `evaluate_classification_error` code here**.

In [1]:
def evaluate_classification_error(tree, data,target='safe_loans'):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error and return it
    # Long winded way - just to show what we trying to achieve!!
    # correct = sum(prediction == data[target])
    # error = len(data) - correct
    # return 1.0*error/len(data)
    
    return 1.0 * sum(prediction != data[target])/len(data)

**20.** Now, let's use this function to evaluate the classification error of `my_decision_tree_new` on the **validation_set**.

In [101]:
round( evaluate_classification_error(my_decision_tree_new, validation_set), 5)

0.38367

**21.** Now, evaluate the validation error using `my_decision_tree_old`.

In [102]:
round(evaluate_classification_error(my_decision_tree_old, validation_set),5)

0.38378

**Quiz question:** Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?  Only a fraction lower!!! We expect it to be lower, cause old tree was overfitting, so whiole its train error wud always be lower, its valid/test error should be higher as it won't be able to generalise well. New tree avoids overfitting when learning the tree, so should be able to generalise better and probbaly achieve lower validation/test errors on unseen data.

# Exploring the effect of max_depth

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**too small**, **just right**, and **too large**).

**2.** Train three models with these parameters:

1. **model_1**: max_depth = 2 (too small)
2. **model_2**: max_depth = 6 (just right)
3. **model_3**: max_depth = 14 (may be too large)

For each of these three, we set `min_node_size = 0` and `min_error_reduction = -1`.

** Note:** Each tree can take up to a few minutes to train. In particular, `model_3` will probably take the longest to train.

In [106]:
model_1 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 2, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Growing right branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth.
########### Grow

In [104]:
model_2 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                 max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

In [117]:
model_3 = my_decision_tree_new = decision_tree_create(train_data, features, 'safe_loans', 
                                 max_depth = 14, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

### Evaluating the models

**23.** Let us evaluate the models on the **train** and **validation** data. Let us start by evaluating the classification error on the training data:

In [110]:
print "Training data, classification error (model 1):", \
     evaluate_classification_error(model_1, train_data)
print "Training data, classification error (model 2):", \
     evaluate_classification_error(model_2, train_data)
print "Training data, classification error (model 3):", \
     evaluate_classification_error(model_3, train_data)

Training data, classification error (model 1): 0.400037610144
Training data, classification error (model 2): 0.381850419084
Training data, classification error (model 3): 0.374462712229


**24.** Now evaluate the classification error on the validation data.

In [112]:
print "Validation data, classification error (model 1):", \
evaluate_classification_error(model_1, validation_set)
print "Validation data, classification error (model 2):", \
evaluate_classification_error(model_2, validation_set)
print "Validation data, classification error (model 3):", \
evaluate_classification_error(model_3, validation_set)

Validation data, classification error (model 1): 0.398104265403
Validation data, classification error (model 2): 0.383778543731
Validation data, classification error (model 3): 0.380008616975


**Quiz Question:** Which tree has the smallest error on the validation data?
Large and complex tree with max_depth=14, model_3

**Quiz Question:** Does the tree with the smallest error in the training data also have the smallest error in the validation data?
Yes, the Large and complex tree with max_depth=14, model_3

**Quiz Question:** Is it always true that the tree with the lowest classification error on the **training** set will result in the lowest classification error in the **validation** set? 
No, in fact we expect the opposite usually. Model or tree with lowest classification error on the **training** set may be overfitting and could result in the higher classification error in the **validation** set.


### Measuring the complexity of the tree

Recall in the lecture that we talked about deeper trees being more complex. We will measure the complexity of the tree as

```
  complexity(T) = number of leaves in the tree T
```

**25.** A function `count_leaves` that counts the number of leaves in a tree. Using this implementation, compute the number of nodes in `model_1`, `model_2`, and `model_3`. 

In [75]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

**26.** Using the function count_leaves, compute the number of nodes in `model_1`, `model_2`, and `model_3`.

In [76]:
print count_leaves(model_1)
print count_leaves(model_2)
print count_leaves(model_3)

4
41
341


**Quiz question:** Which tree has the largest complexity? model_3, the tree with largest num of nodes.
    

**Quiz question:** Is it always true that the most complex tree will result in the lowest classification error in the **validation_set**?   Not always true. Depends on dataset. Generally speaking, the most complex tree will ALWAYS have lowest classification error in the **training_set**, but same complex tree can have the worst/highest classification error in the **validation_set**.

# Exploring the effect of min_error

We will compare three models trained with different values of the stopping criterion. We intentionally picked models at the extreme ends (**negative**, **just right**, and **too positive**).

**27.** Train three models with these parameters:
1. **model_4**: `min_error_reduction = -1` (ignoring this early stopping condition)
2. **model_5**: `min_error_reduction = 0` (just right)
3. **model_6**: `min_error_reduction = 5` (too positive)

For each of these three, we set `max_depth = 6`, and `min_node_size = 0`.

** Note:** Each tree can take up to 30 seconds to train.

In [113]:
model_4 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

In [114]:
model_5 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
Early stopping condition 3 reached. Minimum error reduction.
########### Growing right branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (101 data poi

In [115]:
model_6 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=5.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
Early stopping condition 3 reached. Minimum error reduction.


**28.** Calculate the accuracy of each model (**model_4**, **model_5**, or **model_6**) on the validation set. 

In [116]:
print "Validation data, classification error (model 4):", \
evaluate_classification_error(model_4, validation_set)
print "Validation data, classification error (model 5):", \
evaluate_classification_error(model_5, validation_set)
print "Validation data, classification error (model 6):", \
evaluate_classification_error(model_6, validation_set)

Validation data, classification error (model 4): 0.383778543731
Validation data, classification error (model 5): 0.383778543731
Validation data, classification error (model 6): 0.503446790177


**29.** Using the **`count_leaves`** function, compute the number of leaves in each of each models in (**model_4**, **model_5**, and **model_6**). 

In [118]:
print count_leaves(model_4)
print count_leaves(model_5)
print count_leaves(model_6)

41
13
1


**Quiz Question:** Using the complexity definition above, which model (**model_4**, **model_5**, or **model_6**) has the largest complexity? 
model_4 is most complex - as it has the most num of leaves. Thus many complex tree traversals to form predictions.

Did this match your expectation?  Yes, it had no stopping condition for min error reduction, so all splits were considered, maybe even those that led to increase in cllasification error rate.

**Quiz Question:** **model_4** and **model_5** have similar classification error on the validation set but **model_5** has lower complexity? Should you pick **model_5** over **model_4**? No because error rate over 50% for two class prediction is worse than random guessing!


# Exploring the effect of min_node_size

We will compare three models trained with different values of the stopping criterion. Again, intentionally picked models at the extreme ends (**too small**, **just right**, and **just right**).

**30.** Train three models with these parameters:
1. **model_7**: min_node_size = 0 (too small)
2. **model_8**: min_node_size = 2000 (just right)
3. **model_9**: min_node_size = 50000 (too large)

For each of these three, we set `max_depth = 6`, and `min_error_reduction = -1`.

** Note:** Each tree can take up to 30 seconds to train.

In [119]:
model_7 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 0, min_error_reduction=-1.0)


--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

In [120]:
model_8 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 2000, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).

FOUND LOWER ERROR using grade.A, error is 0.4334, previous best error 10.0000

FOUND LOWER ERROR using term. 36 months, error is 0.4216, previous best error 0.4334
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).

FOUND LOWER ERROR using grade.A, error is 0.3467, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).

FOUND LOWER ERROR using grade.B, error is 0.3463, previous best error 10.0000
########### Growing left branch of tree further ##############
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).

FOUND LOWER ERROR using grade.C, error is 0.3342, prev

In [121]:
model_9 = decision_tree_create(train_data, features, 'safe_loans', 
                                max_depth = 6, min_node_size = 50000, min_error_reduction=-1.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Early stopping condition 2 reached. Reached minimum node size.


**31.** Now, let us evaluate the models (**model_7**, **model_8**, or **model_9**) on the **validation_set**.

In [122]:
print "Validation data, classification error (model 7):", \
evaluate_classification_error(model_7, validation_set)
print "Validation data, classification error (model 8):", \
evaluate_classification_error(model_8, validation_set)
print "Validation data, classification error (model 9):", \
evaluate_classification_error(model_9, validation_set)

Validation data, classification error (model 7): 0.383778543731
Validation data, classification error (model 8): 0.384532529082
Validation data, classification error (model 9): 0.503446790177


**32.** Using the `count_leaves` function, compute the number of leaves in each of each models (**model_7**, **model_8**, and **model_9**). 

In [123]:
print count_leaves(model_7)
print count_leaves(model_8)
print count_leaves(model_9)

41
19
1


**Quiz Question:** Using the results obtained in this section, which model (**model_7**, **model_8**, or **model_9**) would you choose to use? model_8 - just right!!! model_7 is too complex - many nodes, and given min_node_size=0, in theory this can lead to splits resulting in a sample/data point in each node in the worst case - so worst case of overfitting. On the other hand, model_8 has all data in one node - useless for prediction.